# Determinants of Box Office Success

## Imports

In [ ]:
import pandas as pd
import numpy as np
from functools import reduce
from scipy.stats import iqr

# visualization tools
%matplotlib inline
import matplotlib.pyplot as plt

import seaborn as sns

In [ ]:
movies = pd.read_csv('movie_data.csv')
#print(movies.columns)
#print(movies.info())
#movies

**Numeric columns:** num_critic_reviews, duration, director_facebook_likes, actor_1_facebook_likes, actor_2_facebook_likes, actor_3_facebook_likes, movie_facebook_likes, gross, num_users_voted (int), cast_total_facebook_likes  (int), num_user_reviews, budget, title_year , movie_score, aspect_ratio (int)                                                          

## Explore and Clean Dataset

In [ ]:
# remove missing values
movies.dropna(inplace=True)

# remove duplicate rows based on the 'movie_title' column
movies.drop_duplicates(subset=['movie_title'], keep=False, inplace=True)

# reset index
movies.reset_index(drop=True, inplace=True)

In [ ]:
# transform budget and gross metrics
movies['budget_millions'] = round(movies['budget'] / 1000000, 4)
movies['gross_millions'] = round(movies['gross'] / 1000000, 4)

# add calcuated columns
movies['profit_millions'] = movies['gross_millions'] - movies['budget_millions']
movies['roi'] = round((movies['profit_millions'] / movies['budget_millions']), 4)
movies['profit_margin'] = movies['profit_millions'] / movies['gross_millions']

movies['budget_millions'] = movies['budget_millions'].replace(0, np.NaN)
movies['gross_millions'] = movies['gross_millions'].replace(0,np.NaN)

#movies.to_csv('movies_pm_roi.csv')

### Remove irrelevant movies
All movies released prior to 1996 are removed; All non-English language movies are removed

In [ ]:
# remove all movies from before the year 1996 in order to focus on the most recent data points
movies = movies[movies['title_year']>=1996]

# keep only English language movies
movies = movies[movies['language'] == 'English']

movies.reset_index(drop=True, inplace=True)

## Explore Numeric Variables

Several of the numeric variables have significant outliers that will make it more difficult to accurately assess relationships between variables. 

**Solution** - trim numeric variables by replacing outliers with upper whisker values.

**Method** - outliers are identified using the IQR method; replace outliers with upper whisker value; trimmed values will be stored in a separate dataframe (movies_trimmed)

In [ ]:
# define movies trimmed to hold trimmed variables
movies_trimmed = movies.copy(deep = True)

In [ ]:
# separate numerical fields from categorical
movies_numeric = movies.select_dtypes(include=['float64', 'int64'])

# replace outliers with upper whisker / lower whisker value
for column in movies_numeric:
    q1 = movies_trimmed[column].quantile(0.25)
    q3 = movies_trimmed[column].quantile(0.75)
    iqr = q3 - q1
    upper_whisker = q3 + (iqr*1.5)
    lower_whisker = q1 - (iqr*1.5)
    movies_trimmed.loc[movies_trimmed[column] > upper_whisker, column] = upper_whisker
    movies_trimmed.loc[movies_trimmed[column] < lower_whisker, column] = lower_whisker

In [ ]:
#boxplot = movies_trimmed.boxplot(column=['duration'])

In [ ]:
#boxplot = movies_trimmed.boxplot(column=['num_critic_reviews'])

In [ ]:
#boxplot = movies_trimmed.boxplot(column=['director_facebook_likes'])

In [ ]:
#boxplot = movies_trimmed.boxplot(column=['actor_1_facebook_likes'])

In [ ]:
#boxplot = movies_trimmed.boxplot(column=['actor_2_facebook_likes'])

In [ ]:
#boxplot = movies_trimmed.boxplot(column=['actor_3_facebook_likes'])

In [ ]:
#boxplot = movies_trimmed.boxplot(column=['movie_facebook_likes'])

In [ ]:
#boxplot = movies_trimmed.boxplot(column=['cast_total_facebook_likes'])

In [ ]:
#boxplot = movies_trimmed.boxplot(column=['gross_millions'])

In [ ]:
#boxplot = movies_trimmed.boxplot(column=['budget_millions'])

In [ ]:
#boxplot = movies_trimmed.boxplot(column=['movie_score'])

## Explore Categorical Variables

In [ ]:
movies['director_name'].value_counts()

In [ ]:
movies['actor_1_name'].value_counts()

In [ ]:
movies['actor_2_name'].value_counts()

In [ ]:
movies['actor_3_name'].value_counts()

In [ ]:
movies['plot_keywords'].value_counts()

# plot keywords will be more effectively analyzed by separating the keywords

In [ ]:
movies['country'].value_counts()

In [ ]:
movies['content_rating'].value_counts()

In [ ]:
movies['key_actors'] = movies['actor_1_name'].map(str) + '-' + movies['actor_2_name'].map(str) + '-' + movies['actor_3_name'].map(str)
movies['key_actors']

### VAR (Value Above Replacement)
Value Above Replacement (VAR) represents a value of number of times a figure (actor/actress or a director) has exceeded an average amount. In terms of profit, VAR measures the X times an actor/actress or a director appeared in movies with an above average profit.

VAR is calculated by taking the number of times an actor/actress or a director appeared in a movie, dividing it by the net profit of movies they appeared in.

For the sake of this project, we will consider both actors/actresses and directors, with a condition for actor having appeared in at least 5 movies, and directors directing at least 3 movies.

In [32]:
#calculating VAR for actors/actresses

actor1_counts = movies['actor_1_name'].value_counts()

#creating a list with a condition of having appeared in at least 5 movies
actor1_list = actor1_counts[actor1_counts >= 5].index.tolist()

#creating a dataframe with actors in 5 or more movies
actors1 = movies[movies['actor_1_name'].isin(actor1_list)]

#creating a dataframe with actors and the average profit
actor1_total = actors1.groupby(['actor_1_name'], as_index=False)['profit_millions'].mean().sort_values(by='profit_millions', ascending=False)

#creating the VAR value feature
actor1_total['VAR'] = (actor1_total['profit_millions']/actor1_total['profit_millions'].mean())

In [33]:
#calculating VAR for directors

director_counts = movies['director_name'].value_counts()

#creating a list with a condition of having appeared in at least 5 movies
director_list = director_counts[director_counts >= 5].index.tolist()

#creating a dataframe with actors in 5 or more movies
director = movies[movies['director_name'].isin(director_list)]

#creating a dataframe with actors and the average profit
director_total = director.groupby(['director_name'], as_index=False)['profit_millions'].mean().sort_values(by='profit_millions', ascending=False)

#creating the VAR value feature
director_total['VAR'] = (director_total['profit_millions']/director_total['profit_millions'].mean())

## Explore Relationships in the Data

**Variables that are moderately correlated (correlation > 0.4) with gross revenue:** num_critic_reviews, num_users_voted, num_user_reviews, budget, profit_millions, roi, profit_margin 
<br>
**Variables that are moderately correlated (correlation > 0.4) with profit:** gross_millions, num_users_voted, roi, profit_margin
<br>
**Variables that are moderately correlated (correlation > 0.4) with profit margin:** gross_millions, roi, profit_margin
<br>
**Variables that are moderately correlated (correlation > 0.4) with ROI:** gross_millions, profit_millions, profit_margin

In [ ]:
corr_matrix_trimmed = movies_trimmed.corr()
corr_matrix_trimmed.to_csv('corr_matrix_trimmed.csv')

corr_matrix_trimmed

## Success Metrics
Which of the metrics that we have access to demonstrate the success of a film?

Potential measures of success:
1. num_critic_review
2. gross
3. num_user_reviews
4. movie_score

## Analyze director success

Do certain director's tend to make more successful movies?

In [ ]:
#group by director; average gross income
directors_gross = movies[['director_name','gross']].groupby(['director_name']).mean().sort_values(by='gross', ascending=False)
directors_gross['gross_millions'] = round(directors_gross['gross'] / 1000000, 2)

#directors_gross.head(10)

In [ ]:
#group by director; count number of movies
directors_count = movies[['director_name','color']].groupby(['director_name']).count().sort_values(by='color',ascending = False)
directors_count = directors_count.rename(columns = {'color': 'num_movies'})

#directors_count

In [ ]:
#group by director; average movie score
directors_score = movies[['director_name','movie_score']].groupby(['director_name']).mean().sort_values(by='movie_score', ascending = False)

#directors_score.head(10)

In [ ]:
#group by director; average budget
directors_budget = movies[['director_name','budget']].groupby(['director_name']).mean().sort_values(by='budget', ascending = False)
directors_budget['budget_millions'] = round(directors_budget['budget'] / 1000000, 2)

#directors_score.head(10)

In [ ]:
#group by director; average facebook likes
director_fblikes = movies[['director_name','director_facebook_likes']].groupby(['director_name']).mean().sort_values(by='director_facebook_likes', ascending = False)

#director_fblikes.head(10)

In [ ]:
#group by director; average movie facebook likes
movie_fblikes = movies[['director_name','movie_facebook_likes']].groupby(['director_name']).mean().sort_values(by='movie_facebook_likes', ascending = False)

#movie_fblikes.head(10)

In [ ]:
# merge director data into one df
directors_df_lst = [directors_gross, directors_budget, directors_count, directors_score, director_fblikes, movie_fblikes]
directors_df = reduce(lambda left,right: pd.merge(left,right,on='director_name'), directors_df_lst)
directors_df = directors_df.drop(columns=['gross', 'budget'])
directors_df = directors_df.rename(columns = {'gross_millions': 'avg_gross_millions',
                                             'budget_millions': 'avg_budget_million',
                                             'movie_score': 'avg_movie_score',
                                             'director_facebook_likes':'avg_director_fb_likes',
                                             'movie_facebook_likes':'avg_movie_fb_likes'})

directors_df['avg_movie_score'] = round(directors_df['avg_movie_score'],2)
directors_df['avg_movie_fb_likes'] = round(directors_df['avg_movie_fb_likes'],2)
directors_df.to_csv('directors_df.csv')

# filter df to focus on most successful directors
directors_df_filter = directors_df[directors_df['avg_gross_millions']>=100]

In [ ]:
# scatterplot to show relationship between number of movies directed and avg. gross per movie

#sns.set(rc={'figure.figsize':(8,12)})
#sns.set(font_scale=2)
#sns.scatterplot(data=directors_df, x="avg_gross_millions", y="num_movies", s = 70)
#plt.savefig('director_scatter_nummovies_avggross.png')

In [ ]:
# top grossing directors
directors_df_filter.head(10)

In [ ]:
# directors with top average movie scores
directors_df_filter.sort_values(by = 'avg_movie_score', ascending = False).head(10)

In [ ]:
# directors who directed the highest number of movies
directors_df_filter.sort_values(by = 'num_movies', ascending = False).head(10)

## Analzye Genre Success

In [ ]:
movies['genres'].value_counts()

In [ ]:
# create genre dfs
genre_action = movies[movies['genres'].str.contains('Action')]
genre_adventure = movies[movies['genres'].str.contains('Adventure')]
genre_fantasy = movies[movies['genres'].str.contains('Fantasy')]
genre_thriller = movies[movies['genres'].str.contains('Thriller')]
genre_scifi = movies[movies['genres'].str.contains('Sci-Fi')]
genre_crime = movies[movies['genres'].str.contains('Crime')]
genre_comedy = movies[movies['genres'].str.contains('Comedy')]
genre_drama = movies[movies['genres'].str.contains('Drama')]
genre_doc = movies[movies['genres'].str.contains('Documentary')]
genre_romance = movies[movies['genres'].str.contains('Romance')]
genre_horror = movies[movies['genres'].str.contains('Horror')]

In [ ]:
# add binary genre variable to movies df
genre_lst = ['Action','Adventure','Animation','Comedy','Crime','Documentary','Drama','Family','Fantasy','Music','Mystery','Thriller','Horror','Romance','Sci-Fi']

# create column for each genre in genre_lst
for x in genre_lst:
    movies['genre_' + x.lower()] = ""

# populate columns with binary field
for x in range(len(movies['genres'])):
    for y in genre_lst:
        if y in movies['genres'][x]:
            movies['genre_' + y.lower()][x] = y.lower()
        else:
            movies['genre_' + y.lower()][x] = 'not ' + y.lower()

### Create df to analyze average outcome by genre

In [ ]:
genre_avg = pd.DataFrame()
for x in genre_lst:
    x = movies.groupby(['genre_' + x.lower()]).mean()
    x = x.rename_axis('genre', axis= 0)
    genre_avg = genre_avg.append(x)

genre_avg = genre_avg[genre_avg.index.str.contains("not")==False]
genre_avg.to_csv('genre_avg.csv')

#genre_avg

#### barplot - average gross by genre

In [ ]:
genre_avg_gross = sns.barplot(x= genre_avg.index, y="gross_millions", data=genre_avg)
genre_avg_gross.set_xticklabels(genre_avg_gross.get_xticklabels(),rotation=45)
sns.set(rc={'figure.figsize':(20,20)})
sns.set(font_scale=2)
#plt.savefig('genre_avg_gross.png')

#### barplot - average budget by genre

In [ ]:
genre_avg_budget = sns.barplot(x= genre_avg.index, y="budget_millions", data=genre_avg)
genre_avg_budget.set_xticklabels(genre_avg_budget.get_xticklabels(),rotation=45)

#### barplot - average revenue by genre

In [ ]:
genre_avg_revenue = sns.barplot(x= genre_avg.index, y="revenue_millions", data=genre_avg)
genre_avg_revenue.set_xticklabels(genre_avg_revenue.get_xticklabels(),rotation=45)
sns.set(rc={'figure.figsize':(20,15)})
sns.set(font_scale=2)
#plt.savefig('genre_avg_revenue.png')

### Create df to analyzing total outcome (sum) by genre

In [ ]:
genre_sum = pd.DataFrame()
for x in genre_lst:
    x = movies.groupby(['genre_' + x.lower()]).sum()
    x = x.rename_axis('genre', axis= 0)
    genre_sum = genre_sum.append(x)

genre_sum = genre_sum[genre_sum.index.str.contains("not")==False]
#genre_sum.to_csv('genre_sum.csv')

#### barplot - total gross by genre

In [ ]:
#genre_sum_gross = sns.barplot(x= genre_sum.index, y="gross_millions", data=genre_sum)
#genre_sum_gross.set_xticklabels(genre_sum_gross.get_xticklabels(),rotation=45)
#sns.set(rc={'figure.figsize':(20,15)})
#sns.set(font_scale=2)
#plt.savefig('genre_sum_gross.png')

#### barplot - total budget by genre

In [ ]:
#genre_sum_budget = sns.barplot(x= genre_sum.index, y="budget_millions", data=genre_sum)
#genre_sum_budget.set_xticklabels(genre_sum_budget.get_xticklabels(),rotation=45)

#### barplot - total revenue by genre

In [ ]:
#genre_sum_revenue = sns.barplot(x= genre_sum.index, y="revenue_millions", data=genre_sum)
#genre_sum_revenue.set_xticklabels(genre_sum_revenue.get_xticklabels(),rotation=45)
#sns.set(rc={'figure.figsize':(20,15)})
#sns.set(font_scale=2)
#plt.savefig('genre_sum_revenue.png')